#### Query Agent Demo

In [ ]:
import openai
import re
import os
import logging
import sys
from logging import StreamHandler


In [ ]:
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
  
# Configure environment variables  
az_search_service_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
az_search_index_name = os.environ["AZURE_SEARCH_INDEX_NAME"]
az_search_credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"])

In [ ]:
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_version = os.environ["OPENAI_API_VERSION"]
text_gen_deploy_name = os.environ["OPENAI_API_TEXT_GEN_DEPLOY"]
embeddings_deploy_name = os.environ["OPENAI_API_EMBEDDINGS_DEPLOY"]


In [ ]:
print(text_gen_deploy_name)
print(az_search_index_name)
print(az_search_service_endpoint)
print(az_search_credential)
print(embeddings_deploy_name)

In [ ]:
nl_queries = [
    "Who are my most valuable customers",                               #0
    "Give me 10 products",                                              #1
    "Which products are sold most in NY and PA",                        #2
    "Which products are sold most in Newyork and Pennsylvania",         #3
    "I need top selling products",                                      #4
    "How many customers are there in NY",                               #5
    "How many customers are there in Utah",                             #6
    "How many customers are there in UT"                                #7
]

In [ ]:
query = nl_queries[0]
search_client = SearchClient(az_search_service_endpoint, 
                             az_search_index_name, 
                             credential=az_search_credential)  

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text):
    response = openai.Embedding.create(input=text, engine=embeddings_deploy_name)
    embeddings = response['data'][0]['embedding']
    return embeddings

results = search_client.search(  
    search_text=None,  
    vector=generate_embeddings(query),
    top_k=3,  
    vector_fields="NLQueryVector",
    select=["NLQuery", "KQLQuery"],
)  

results = [{ 
                "NLQuery": result["NLQuery"], 
                "KQLQuery": result["KQLQuery"]
            } for result in list(results)]
# print(json.dumps(results))

In [ ]:
with open("./data/kql_reference.json", 'r') as language_ref_file, \
     open("./data/database_schema.json", 'r') as db_schema_file, \
     open("./data/kql_examples.json", 'r') as examples_file:
    language_reference = language_ref_file.read().strip()
    database_schema = db_schema_file.read().strip()
#     translation_examples = examples_file.read().strip()
    translation_examples = json.dumps(results)
    

In [ ]:
print(database_schema)

In [ ]:
session_examples=f"""
Give me the total number of products
Thought: I need to count all rows from products table.
OutputQuery:
products
| count
WAITING

You will be then called again with the results of query execution:
Results:
77

You will then output the following:
Thought: Query is returning rows, there are no errors apparently. Therefore it is correct. 
OutputQuery:
products
| count
""".strip()

In [ ]:
delimiter='####'
prompt=f"""As an expert in Kusto Query Language, your task is to translate natural language queries into KQL queries.
The process will proceed through a series of steps in a loop until an accurate query is obtained.

To accomplish this task, you will utilize:

1. Sample Translations: This helps to understand how input query can be translated into KQL for the specific database your are querying: 
{translation_examples}
2. Database Schema Reference: This helps to determine the tables and columns that need to be used in the output query: 
{database_schema}
3. Language Reference: This helps to understand the syntax of the output query: 
{language_reference}

You will output the steps using the following format:

Thought:  You will describe your thought process on how to translate the input query into KQL.
OutputQuery:  You will generate the output query.
WAITING:  You will return WAITING.

Results: This is the result of the query execution(first few rows only) which will be provided to you in the next call;\
You will not generate this yourself.

You will analyze the results for errors and inaccuracies, if it is correct, you will generate final output in the following format:
Thought: Your thought process about the results. 
FinalQuery: the latest output query generated. You will not include Results or OutputQuery in the final output.

Here are some example sessions: 
{session_examples}
""".strip()


In [ ]:
print(prompt)

In [ ]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd

class KQLTools:
    def __init__(self, cluster, database) -> None:
        aad_tenant_id = os.environ['AAD_TENANT_ID']
        aad_client_id = os.environ['AAD_CLIENT_ID']
        aad_client_secret = os.environ['AAD_CLIENT_SECRET']
        kscb = KustoConnectionStringBuilder.with_aad_application_key_authentication(cluster, 
                                                    aad_client_id,
                                                    aad_client_secret, 
                                                    aad_tenant_id)
        self.client = KustoClient(kscb)
        self.database = database

    def execute_query(self, query):
        response  = self.client.execute(self.database, query)
        result_df = dataframe_from_result_table(response.primary_results[0])
        return result_df


In [ ]:

class LLMChat:
    def __init__(self, system_prompt="", debug=False):
        self.messages = []
        self.debug = False
        self.messages.append({"role": "system", "content": system_prompt})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.run()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def run(self, debug=False):        
        completion = openai.ChatCompletion.create(engine=text_gen_deploy_name, 
                                                  temperature=0,
                                                  messages=self.messages)
        if (self.debug):
            {"completion_tokens": 86, "prompt_tokens": 26, "total_tokens": 112}
            print(completion.usage)        
        return completion.choices[0].message.content


In [ ]:
def setup_logging(debug=False):
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG if debug else logging.INFO)

    # Remove all handlers associated with the logger object by default.
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)

    # Create a console handler
    ch = logging.StreamHandler(sys.stdout)
    ch.setLevel(logging.DEBUG if debug else logging.INFO)

    # Create formatter and add it to the handlers
    formatter = logging.Formatter('%(message)s')
    ch.setFormatter(formatter)

    # Add the handlers to the logger
    logger.addHandler(ch)

In [ ]:
class QueryAgent(LLMChat):
    def __init__(self, system_prompt, db_tools, max_tries=5, debug=False):
        super().__init__(system_prompt, debug)        
        self.max_tries = max_tries
        self.db_tools = db_tools
        self.logger = logging.getLogger(__name__)

    # def extract_query(self, result):
    #     return [self.query_re.match(a) for a in result.split('\n') if self.query_re.match(a)]

    def extract_query(self, text):
        pattern_final = re.compile(r'FinalQuery:(.*?)(?:(?=\nOutputQuery:)|(?=$))', re.DOTALL)
        matches_final = pattern_final.findall(text)
        
        if matches_final:
            return 'Final', matches_final[0].strip()
        
        pattern_output = re.compile(r'OutputQuery:(.*?)(?:(?=\nWAITING)|(?=\nOutputQuery:)|(?=$))', re.DOTALL)
        matches_output = pattern_output.findall(text)

        return 'Output', matches_output[-1].strip() if matches_output else None    
    

    def query(self, input_query, sample_rows_num=5):
        next_prompt = input_query
        self.logger.info(next_prompt)
        for i in range(self.max_tries):
            llm_result = super().__call__(next_prompt)
            self.logger.info(llm_result)
            query_type, query_text = self.extract_query(llm_result)
            if query_type == 'Output':
                # print("kql\n" + query_text)
                try: 
                    query_results=self.db_tools.execute_query(query_text)
                    next_prompt = f"Results: {query_results.head(sample_rows_num).to_csv()}"
                except Exception as e:
                    next_prompt = f"Results: {e}"
                # self.logger.info(next_prompt)
            else:
                return query_text


In [ ]:
setup_logging(debug=True)
kusto_cluster = "https://dxp01.adnocpocsynapse.kusto.azuresynapse.net"
kusto_database = "retail_org"
kql_tools = KQLTools(kusto_cluster, kusto_database)

kql_agent = QueryAgent(prompt,kql_tools, max_tries=3, debug=True)
kql_query=kql_agent.query(query)

In [ ]:
print(kql_query)
print(kql_tools.execute_query(kql_query))